In [ ]:
from mongo import get_collection

DB_HOST = "localhost"
collection = get_collection(DB_HOST)

In [ ]:
import pandas as pd

# Récupérer tous les documents de la collection
documents = list(collection.find())

# Convertir les documents en DataFrame Pandas
df_collections = pd.DataFrame(documents)

# Afficher le DataFrame
df_livres = df_collections[df_collections.type == "livres"]

In [ ]:
oid_list = df_livres["_id"].tolist()

In [ ]:
from bson import ObjectId


def get_transcription(collection, oid, verbose=False):
    # Récupération du document
    document_entry = collection.find_one({"_id": ObjectId(oid)})

    if document_entry is None:
        if verbose:
            print(f"Document avec l'oid {oid} non trouvé")
        return None

    return document_entry.get("whisper")

In [ ]:
def get_prompt(transcription):
    prompt = f"""

    Je vais te donner la transcription d'un episode d'une emission de radio qui s'appelle le masque et la plume sur France Inter.
    Cet episode dure 1h et porte sur des livres. Il y a des intervenants qui parlent des livres qu'ils ont lus. Ils ne sont parfois pas d'accord.

    Voici la transcription:
    {transcription}

    Je veux que tu identifies l'ensemble des livres dont on parle dans cette emission.
    Et que tu me restitues cette liste de livres en separant auteur et titre. Si l'editeur est mentionne tu peux aussi le noter.
    Tu me restitueras cette liste sous la forme d'un tableau au format markdown.

    Ensuite je veux un 2e tableau qui me donne les avis des critiques sur ces livres.
    Concernant les avis des critiques, je veux que tu en fasses une forme de synthese en donnant une note de 1 à 10 (1 etant la note la plus basse et 10 la note la plus haute) utilisant les regles suivantes:
    - la note 1 est vraiment pour les livres a eviter, les purges
    - la note 10 est pour les livres a lire absolument, les chefs d'oeuvre
    - la note 9 est pour les livres excellents, 
    - la note 5 est pour les livres moyens, sans plus. pas horrible mais pas genial non plus
    - les notations seront assez severes, ne garde la note 10 vraiment que pour les chefs d'oeuvre
    - si un seul critique donne son avis, tu prendras sa note
    - si plusieurs critiques se prononcent, tu prendras la moyenne de leurs notes
    Je veux que tu conserves l'avis de chaque critique avec son prenom et son nom.
    et que tu donnes la note moyenne obtenue pour chaque livre.
    tu rajouteras une colonne pour dire le nombre de critiques qui ont donne leur avis sur le livre.
    Puis si un des critiques a vraiment adore le livre (ce qui correspond a une note de 9 ou 10), tu mentionneras les noms des critiques dans une colonne "coup de coeur" a part.
    Enfin si un des critiques fait etat d'un chef d'oeuvre (note 10), tu mentionneras cela dans une colonne "chef d'oeuvre" a part.
    Tu me restitueras cette liste sous la forme d'un tableau au format markdown.

    """
    return prompt

In [ ]:
from llm import get_gemini_llm

llm = get_gemini_llm()

for oid in oid_list:
    raw_oid = str(oid)
    transcription = get_transcription(collection, raw_oid)
    response = llm.generate_content(get_prompt(transcription))
    print(raw_oid)
    print(response.text)

6773e32258fc5717f3516b99
Voici les deux tableaux demandés, synthétisant les informations extraites de la transcription de l'émission "Le Masque et la Plume".


**Tableau 1: Liste des livres présentés**

| Auteur             | Titre                                      | Éditeur              |
|----------------------|---------------------------------------------|-----------------------|
| Marc Dugain          | L'avion, Poutine, l'Amérique et moi         | Albin Michel           |
| Emmanuel Lambert     | Aucun respect                               | Stock                 |
| Emile Ferris        | Moi, ce que j'aime, c'est les monstres (Tome 2)| Monsieur Toussaint L'Ouverture |
| Fabrice Carreau      | Fort Alamo                                  | Gallimard              |
| Mathieu Palin        | Les Hommes Manquent de Courage             | L'iconoclaste          |
| H.P. Lovecraft      | Œuvres (édition Pléiade)                   | Gallimard              |
| Xavier Lardou        | Chri